In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_df = pd.read_csv("./train.csv")
train_df["datetime"] = pd.to_datetime(train_df["timestamp"])
train_df["year"] = train_df["datetime"].dt.year
train_df["month"] = train_df["datetime"].dt.month
train_df["day"] = train_df["datetime"].dt.day

In [3]:
temp = train_df['build_year'].copy()
temp[train_df["build_year"] > 1000] = train_df["year"][train_df["build_year"] > 1000] - train_df["build_year"][train_df["build_year"] > 1000]
train_df["build_year"] = temp

In [4]:
train_df["product_type"] = train_df["product_type"].replace("Investment", -1)
train_df["product_type"] = train_df["product_type"].replace("OwnerOccupier", 1)
train_df["product_type"].value_counts()

-1    19448
 1    11023
Name: product_type, dtype: int64

In [6]:
test_df = pd.read_csv("./test.csv")
test_df["datetime"] = pd.to_datetime(test_df["timestamp"])
test_df["year"] = test_df["datetime"].dt.year
test_df["month"] = test_df["datetime"].dt.month
test_df["day"] = test_df["datetime"].dt.day
temp2 = test_df['build_year'].copy()
temp2[test_df["build_year"] > 1000] = test_df["year"][test_df["build_year"] > 1000] - test_df["build_year"][test_df["build_year"] > 1000]
test_df["build_year"] = temp2
test_df["product_type"] = test_df["product_type"].replace("Investment", -1)
test_df["product_type"] = test_df["product_type"].replace("OwnerOccupier", 1)
test_df["product_type"].value_counts()

-1.0    4998
 1.0    2631
Name: product_type, dtype: int64

In [7]:
train_df = train_df.replace({"no": -1, "yes": 1})
test_df = test_df.replace({"no": -1, "yes": 1})

In [8]:
inplace1 = (train_df["full_sq"] < train_df["life_sq"]) & (train_df["full_sq"] < 10)

In [10]:
inplace1.value_counts()

False    30464
True         7
dtype: int64

In [11]:
temp3 = train_df["full_sq"].copy()
temp3[inplace1] = train_df["life_sq"][inplace1]
train_df["full_sq"] = temp3

In [12]:
train_df["full_sq"].value_counts()

38      1928
37      1232
39      1013
44      1009
45       943
40       862
32       846
43       832
41       764
51       750
42       695
53       665
52       654
60       636
63       628
34       614
35       613
64       609
58       576
59       575
54       567
36       544
61       543
31       516
46       479
62       474
79       470
57       466
74       466
56       447
        ... 
637        1
461        1
189        1
173        1
5326       1
412        1
204        1
603        1
11         1
394        1
218        1
186        1
10         1
729        1
185        1
216        1
184        1
168        1
207        1
407        1
215        1
199        1
182        1
6          1
325        1
388        1
291        1
275        1
195        1
0          1
Name: full_sq, Length: 209, dtype: int64

In [23]:
inplace2 = (train_df["full_sq"] > 30) & (train_df["life_sq"] < 10)
inplace2.value_counts()

False    30060
True       411
dtype: int64

In [24]:
temp4 = train_df["life_sq"].copy()
temp4[inplace2] = train_df["full_sq"][inplace2]
train_df["life_sq"] = temp4

In [28]:
inplace3 = (train_df["full_sq"] < train_df["life_sq"]) & (train_df["life_sq"] > 10)
inplace3.value_counts()

False    30441
True        30
dtype: int64

In [29]:
temp5 = train_df["life_sq"].copy()
temp5[inplace3] = train_df["full_sq"][inplace3]
train_df["life_sq"] = temp5

In [30]:
inplace1 = (test_df["full_sq"] < test_df["life_sq"]) & (test_df["full_sq"] < 10)
temp3 = test_df["full_sq"].copy()
temp3[inplace1] = test_df["life_sq"][inplace1]
test_df["full_sq"] = temp3
inplace2 = (test_df["full_sq"] > 30) & (test_df["life_sq"] < 10)
temp4 = test_df["life_sq"].copy()
temp4[inplace2] = test_df["full_sq"][inplace2]
test_df["life_sq"] = temp4
inplace3 = (test_df["full_sq"] < test_df["life_sq"]) & (test_df["life_sq"] > 10)
temp5 = test_df["life_sq"].copy()
temp5[inplace3] = test_df["full_sq"][inplace3]
test_df["life_sq"] = temp5

In [31]:
y_train = train_df['price_doc'].values
id_test = test_df['id']
train_df.drop(['timestamp'], axis=1, inplace=True)
train_df.drop(['year'], axis=1, inplace=True)
train_df.drop(['month'], axis=1, inplace=True)
train_df.drop(['day'], axis=1, inplace=True)
test_df.drop(['timestamp'], axis=1, inplace=True)
test_df.drop(['year'], axis=1, inplace=True)
test_df.drop(['month'], axis=1, inplace=True)
test_df.drop(['day'], axis=1, inplace=True)
train_df.drop(['datetime'], axis=1, inplace=True)
test_df.drop(['datetime'], axis=1, inplace=True)
train_df["ecology"] = train_df["ecology"].replace("poor", -1)
train_df["ecology"] = train_df["ecology"].replace("no data", 0)
train_df["ecology"] = train_df["ecology"].replace("satisfactory", 1)
train_df["ecology"] = train_df["ecology"].replace("good", 2)
train_df["ecology"] = train_df["ecology"].replace("excellent", 3)
test_df["ecology"] = test_df["ecology"].replace("poor", -1)
test_df["ecology"] = test_df["ecology"].replace("no data", 0)
test_df["ecology"] = test_df["ecology"].replace("satisfactory", 1)
test_df["ecology"] = test_df["ecology"].replace("good", 2)
test_df["ecology"] = test_df["ecology"].replace("excellent", 3)

In [48]:
train_df.drop("id", axis=1, inplace=True)
train_df.drop("price_doc", axis=1, inplace=True)

In [49]:
test_df.drop("id", axis=1, inplace=True)

In [50]:
test_df.drop("sub_area", axis=1, inplace=True)

ValueError: labels ['sub_area'] not contained in axis

In [51]:
num_boost_rounds = 489

In [52]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(train_df, y_train)
dtest = xgb.DMatrix(test_df)

In [53]:
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= num_boost_rounds)

In [58]:
y_predict = model.predict(dtest)
output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})

In [59]:
output.head()

,id,price_doc
0,30474,5647447.5
1,30475,8186291.0
2,30476,5522518.5
3,30477,5822495.5
4,30478,5207038.5


In [60]:
output.to_csv('TryAgain7.csv', index=False)